# DropOut

Podme implementovat DropOut. Pouzijeme metodu "Inverted dropout".

In [3]:
import numpy as np

from backstage.layer import Layer


## 1. Konstruktor

Parametrom vrstvy DropOut je __dropRate__ - sanca, ze neuron bude deaktivovany. Cize __dropRate = 0__ znamena pouzitie vsetkych neuronov, a __dropRate = 0.8__ znamena, ze kazdy neuron ma 80% sancu ze bude vypnuty - pouzijeme iba 20% neuronov.

In [4]:
class DropOut(Layer):
    def __init__(self, dropRate=0.5):
        super().__ini__(act='linear')

        # Inverted dropout znamena, ze pracujeme s 
        #    keepProb - pravdepodobnostou ponechania neuronov
        self.keepProb = 1.0 - dropRate

## 2. Inicializacia

Dropout vrstva ma rovnaky tvar ako predcahdzajuca vrstva.

In [5]:
class DropOut(DropOut):

    def initialize(self, prevLayer):

        # Pouzivame tvar predchadzajucej vrstvy
        self.shape = prevLayer.shape

## 3. Priamy prechod

DropOut je prvy typ vrstvy, ktory sme stretli, ktory ma rozdielne spravanie pocas ucenia a pocas predikcie. Preto rozsirujeme metodu __forward__ o argument __isTraining__.

Vypocitame si __mask__ - boolean masku, ktora bude mat hodnoty True alebo False podla toho, ci si dany neuron chceme ponechat, alebo ho deaktivovat. Metoda Inverted Dropout pouziva este korekciu skalovania, ktora zaruci, ze celkovy objem aktivacie zostane zachovany.

In [6]:
class DropOut(DropOut):

    # Novy argument - isTraining
    def forward(self, x, isTraining):

        if (isTraining):
            # Drop masku vzdy pri kazdom prechode a aplikujeme korekciu skalovania
            mask = (np.random.rand(*x.shape) < self.keepProb)
            mask = dropMask / self.keepProb

            # Masku budeme potrebovat pri spatnom prechode
            return x*mask, mask

        # Pocas predikcie vratime priamo x
        return x, None


## 4. Spatny prechod

Pouzijeme masku, ktoru sme vratili ako nacachovanu hodnotu. A gradient aktivacie sirime iba cez tie neurony, ktore v priamom prechode zostali aktivne.

DropOut nema ziadne parametre, ktore by sa mohli ucit, preto s optimizerom nerobime nic.


In [7]:
class DropOut(DropOut):

    def backward(self, da, aPrev, cache, optimizer):

        # Masku sme nacachovali pri priamom prechode
        mask = cache

        # Gradient sirime iba pre aktivne neurony z priameho prechodu
        daPrev = da * mask
        return daPrev